In [1]:
import pandas as pd
import numpy as np

In [2]:
# !pip install apache-superset

In [31]:
# !pip install cassandra-driver

In [32]:
import cassandra
# print(cassandra.*version*)

In [33]:
csv_file_path = "/Users/lavanyamk/Documents/FeaViz/Olist_ecommerce_data/olist_products_dataset_output.csv"
db_bundle_path = "/Users/lavanyamk/Downloads/secure-connect-nosql-db.zip"
user_name = "CgYxKnkrerLwiDCMYlNXdPHa"
password = "91YcRxuTwnZR-.WB-TJa5HOoFL8oL,AdmQusxObsSK5pIriJM0kFX9oSgr1jj3BakDZ5jOz4AZy0rpOwT76JWUNQWTC8u1an5FmJs-FR1xsczTLxJXFGUZczuGCMJ9aK"

In [6]:
# %env PYSPARK_SUBMIT_ARGS=' --packages com.datastax.spark:spark-cassandra-connector_2.12:3.0.1'

In [34]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [7]:
import findspark
findspark.init()
import pyspark

In [8]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, from_unixtime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer


spark = SparkSession.builder.appName("temp").getOrCreate()


# spark = SparkSession \
#     .builder \
#     .appName('demo') \
#     .master("local") \
#     .config( \
#         "spark.cassandra.connection.config.cloud.path", \
#         "file:"+db_bundle_path) \
#     .config("spark.cassandra.auth.username", user_name) \
#     .config("spark.cassandra.auth.password", password) \
#     .getOrCreate()

In [9]:
# spark.sparkContext.version
!echo $SPARK_HOME

/opt/spark-3.0.1-bin-hadoop3.2


In [10]:
spark.sparkContext.version

'3.0.1'

In [10]:
df = spark.read.format("org.apache.spark.sql.cassandra"
                            ).options(table="users_by_city", keyspace="nosql1").load()

In [13]:
# TODO: Upload csv to AstraDB
# TODO: Read from AstraDB
features_df = spark.read.option("inferSchema", "true").csv(
    "file://"+csv_file_path, 
    header=True)

In [15]:
from pyspark.sql import Row
from collections import OrderedDict

In [16]:
def get_transpose_df(df, columns, pivotCol):
    columnsValue = list(map(lambda x: str("'") + str(x) + str("',")  + str(x), columns))
    stackCols = ','.join(x for x in columnsValue)
    df_1 = df.selectExpr(pivotCol, "stack(" + str(len(columns)) + "," + stackCols + ")")\
           .select(pivotCol, "col0", "col1")
    final_df = df_1.groupBy(col("col0")).pivot(pivotCol).agg(concat_ws("", collect_list(col("col1"))))\
                 .withColumnRenamed("col0", pivotCol)
    return final_df

In [17]:
features_df.toPandas()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [18]:
features_df.printSchema

<bound method DataFrame.printSchema of DataFrame[product_id: string, product_category_name: string, product_name_lenght: double, product_description_lenght: double, product_photos_qty: double, product_weight_g: double, product_length_cm: double, product_height_cm: double, product_width_cm: double]>

In [19]:
# tuple([str(m[1]) for m in features_df.dtypes]), features_df.columns

In [20]:
#ref: https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/
# spark.createDataFrame(features_df.dtypes).show()

# spark.createDataFrame([("data_type")] + [(m[1]) for m in features_df.dtypes], ["summary"] + features_df.columns)
# rdd.toDF(columns).show()

In [51]:

# data

In [56]:
def create_feature_summary(features_df):
    distict_count_summary_col = lit("distinct_count").alias("summary")
    missing_count_summary_col = lit("missing_count").alias("summary")
    exprs = [distict_count_summary_col] + [countDistinct(x).alias(x) for x in features_df.columns]

    data ={'summary':'data_type'}
    data.update( {m[0]:m[1] for m in features_df.dtypes})


    distinct_count_df = features_df.agg(*exprs)
    missing_df = features_df.select([missing_count_summary_col] +
                       [count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in features_df.columns])
    dtype_df = spark.createDataFrame([data])

    custom_summary_df = dtype_df.unionByName(missing_df).unionByName(distinct_count_df)

    summary_t_df = features_df.describe().unionByName(features_df.summary("25%", "50%", "75%")).unionByName(custom_summary_df)
    summary_df = get_transpose_df(summary_t_df, features_df.columns, "summary").withColumnRenamed("25%", "top_25"
                                ).withColumnRenamed("50%", "top_50"
                                                   ).withColumnRenamed("75%", "top_75")
    return summary_df

In [57]:
summary_df = create_feature_summary(features_df)
summary_df.show()

+--------------------+------+------+------+-----+---------+--------------+--------------------+------------------+--------------------+-------------+------------------+
|             summary|top_25|top_50|top_75|count|data_type|distinct_count|                 max|              mean|                 min|missing_count|            stddev|
+--------------------+------+------+------+-----+---------+--------------+--------------------+------------------+--------------------+-------------+------------------+
|  product_photos_qty|   1.0|   1.0|   3.0|32341|   double|            19|                20.0|2.1889861166939797|                 1.0|          610|1.7367656379315435|
|product_category_...|      |      |      |32341|   string|            73|       watches_gifts|                  |agro_industry_and...|          610|                  |
|    product_weight_g| 300.0| 700.0|1900.0|32949|   double|          2204|             40425.0|2276.4724877841513|                 0.0|            2| 4282.

In [ ]:
# Write to AstraDB with the same DDL of the above table but table is input_file_name_feature_summary

In [55]:
summary_df.coalesce(1).write.format('com.databricks.spark.csv').option(
    "header",True) .mode('overwrite').csv("file:///Users/lavanyamk/Documents/FeaViz/outputs/feature_summary.csv")
# Write to AstraDB with table nam

In [35]:

cloud_config= {
        'secure_connect_bundle': db_bundle_path
}
auth_provider = PlainTextAuthProvider(user_name, password)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select * from nosql1.users_by_city").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

PARIS


In [39]:
print([col for col in summary_df.dtypes])

[('summary', 'string'), ('product_id', 'string'), ('product_category_name', 'string'), ('product_name_lenght', 'string'), ('product_description_lenght', 'string'), ('product_photos_qty', 'string'), ('product_weight_g', 'string'), ('product_length_cm', 'string'), ('product_height_cm', 'string'), ('product_width_cm', 'string')]


In [ ]:
# Now both input data & summary data is available AstraDB
# Create empty dashboard

# Create Feature Summary in Table View
    # VISUALIZATION TYPE -> Table
    # QUERY MODE -> RAW RECORDS
    # COLUMNS -> Check for all columns options
# Add this table with full width to dashboard

# TODO: Read this dtypes & call superset API to create individual column viz
# for i in dtypes:
    # Create charts
    # Numerical (Double, Int) -> VISUALIZATION TYPE Histogram, COLUMNS is the col_name  ex: product_height_cm_count
    # String -> VISUALIZATION TYPE Histogram, COLUMNS is the col_name  ex: product_height_cm_count
        # VISUALIZATION TYPE -> Bar Chart
        # METRICS -> COUNT(*)
        # Series -> col_name

# Add all charts to prev dashboard. Each chart has width 1/3.


In [ ]:
session.execute("")

In [28]:
# imporpyspark.sql.catalogca

In [29]:
summary_df.write.format("org.apache.spark.sql.cassandra"
                        ).mode('append'
                              ).options(table="feature_summary", keyspace="nosql1").save()

AnalysisException: Couldn't find nosql1.feature_summary or any similarly named keyspace and table pairs;

In [76]:
features_df.dtypes

[('product_id', 'string'),
 ('product_category_name', 'string'),
 ('product_name_lenght', 'double'),
 ('product_description_lenght', 'double'),
 ('product_photos_qty', 'double'),
 ('product_weight_g', 'double'),
 ('product_length_cm', 'double'),
 ('product_height_cm', 'double'),
 ('product_width_cm', 'double')]

+--------------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|       summary|         review_id|            order_id|        review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|         count|            105188|              102859|              102692|               12176|                 41868|               96025|                  96002|
|          mean|               4.5|                 0.0|   4.071667849964501|3.155449396525236...|  1.111111111111111...|                null|                   null|
|        stddev|0.7071067811865476|                 0.0|   1.386648877434681|5.616554832455847E11|              Infinity|                null|                   null

In [100]:
features_df.agg(countDistinct("product_id").alias("total_count")).show()

+-----------+
|total_count|
+-----------+
|      32951|
+-----------+



In [74]:
describe_features_df = get_transpose_df(features_df.describe(), features_df.columns, "summary")
describe_features_df.show()

+--------------------+-----+--------------------+------------------+--------------------+------------------+
|             summary|count|                 max|              mean|                 min|            stddev|
+--------------------+-----+--------------------+------------------+--------------------+------------------+
|  product_photos_qty|32341|                   9|2.1889861166939797|                   1|1.7367656379315435|
|product_category_...|32341|utilidades_domest...|                  |agro_industria_e_...|                  |
|    product_weight_g|32949|                 998|2276.4724877841513|                   0| 4282.038730977024|
|    product_width_cm|32949|                  98|23.196728277034204|                  10|12.079047453227794|
| product_name_lenght|32341|                   9| 48.47694876472589|                  10|10.245740725237287|
|   product_height_cm|32949|                  99|16.937661234028347|                  10|13.637554061749569|
|   product_length_

In [90]:
@udf(returnType=StringType())
def get_distinct_count(col_name_str, features_df):
    distinct_count = features_df.select(col_name_str).distinct().count()
    return distinct_count 

# get_distinct_count_udf = udf(lambda z: get_distinct_count(z),IntegerType())

In [91]:
features_df.select("product_photos_qty").distinct().count()

20

In [92]:
describe_features_df.select("summary")

DataFrame[summary: string]

In [93]:
describe_features_df.withColumn("type", get_distinct_count_udf(col("summary"), features_df))#.show(truncate=False)

# df.select(col("Seqno"), \
#     convertUDF(col("Name")).alias("Name") ) \
#    .show(truncate=False)

TypeError: Invalid argument, not a string or column: DataFrame[product_id: string, product_category_name: string, product_name_lenght: string, product_description_lenght: string, product_photos_qty: string, product_weight_g: string, product_length_cm: string, product_height_cm: string, product_width_cm: string] of type <class 'pyspark.sql.dataframe.DataFrame'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [ ]:
# from pyspark.sql.functions import *
# from pyspark.sql import SparkSession

# def main():
#     DataList = [("Shirts", 10, 13, 34, 10), ("Trousers", 11, 2, 30, 20), ("Pants", 70, 43, 24, 60), ("Sweater", 101, 44, 54, 80)]
#     productQtyDF  = spark.createDataFrame(DataList, ["Products", "Small", "Medium", "Large", "ExLarge"])
#     productQtyDF.show()
#     #+--------+-----+------+-----+-------+
#     #|Products|Small|Medium|Large|ExLarge|
#     #+--------+-----+------+-----+-------+
#     #|  Shirts|   10|    13|   34|     10|
#     #|Trousers|   11|     2|   30|     20|
#     #|   Pants|   70|    43|   24|     60|
#     #| Sweater|  101|    44|   54|     80|
#     #+--------+-----+------+-----+-------+

#     productTypeDF = TransposeDF(productQtyDF, ["Small", "Medium", "Large", "ExLarge"], "Products")
#     productTypeDF.show(truncate= False)
#     #+--------+-----+------+-------+--------+
#     #|Products|Pants|Shirts|Sweater|Trousers|
#     #+--------+-----+------+-------+--------+
#     #|Medium  |43   |13    |44     |2       |
#     #|Small   |70   |10    |101    |11      |
#     #|ExLarge |60   |10    |80     |20      |
#     #|Large   |24   |34    |54     |30      |
#     #+--------+-----+------+-------+--------+


# if __name__ == '__main__':
#     main()

### Read Data

In [45]:
df.count()

32952

### Print schema

In [46]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string]>

In [47]:
df.show(1)

+----------+--------------------+-------------------+--------------------+------------------+----------------+-----------------+-----------------+----------------+
|       _c0|                 _c1|                _c2|                 _c3|               _c4|             _c5|              _c6|              _c7|             _c8|
+----------+--------------------+-------------------+--------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_...|product_name_lenght|product_descripti...|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+--------------------+-------------------+--------------------+------------------+----------------+-----------------+-----------------+----------------+
only showing top 1 row

